In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00


In [19]:
import numpy as np
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Load your dataset (replace 'reviews.csv' with your dataset path)
# Read the CSV containing reviews for multiple places
df = pd.read_csv('/content/reviews (1).csv', sep=',')

# Data preprocessing
def preprocess_text(text):
    # Add your text preprocessing steps here
    # Example: Remove HTML tags, URLs, punctuation, and lowercase text
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'http\S+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    return text

df['A Detailed Review of the Place'] = df['A Detailed Review of the Place'].apply(preprocess_text)

# Map ratings to sentiment labels
def map_rating_to_sentiment(rating):
    if rating <= 2:
        return "Bored"
    elif rating == 3:
        return "Mixed emotions"
    elif rating >= 4:
        return "Happy"

df['sentiment'] = df['On a Scale of 1-5 Rate the Place'].apply(map_rating_to_sentiment)

# Encode labels as integers
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

# Initialize DistilBERT tokenizer
max_seq_length = 128
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', max_length=max_seq_length, truncation=True, padding='max_length')

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['Place', 'Accuracy', 'Classification Report'])

place_sentiments ={'Charminar','Golconda Fort', 'Wonderla', 'Ramoji Film City', 'Nehru Zoological Park', 'Birla Science Museum', 'Hussain Sagar Lake', 'Birla mandir', 'Cable Bridge', 'NTR Garden'}

model_tot={}
y_pred_tot={}
accuracy_tot={}
classification_rep_tot={}

# Loop through each place in the dataset
for place in df['Place You Want to Review About'].unique():
    print(f"Processing reviews for Place: {place}")

    # Filter the DataFrame to select reviews for the current place
    place_reviews = df[df['Place You Want to Review About'] == place]

    # Train-test split for the specific place
    x_train, x_test, y_train, y_test = train_test_split(
        place_reviews['A Detailed Review of the Place'],
        place_reviews['sentiment'],
        test_size=0.3,
        random_state=42,
        # stratify=place_reviews['sentiment']
    )

    # Tokenize and encode the training data
    x_train_encoded = tokenizer(list(x_train), return_tensors='tf', padding=True, truncation=True, max_length=max_seq_length)

    # Tokenize and encode the test data
    x_test_encoded = tokenizer(list(x_test), return_tensors='tf', padding=True, truncation=True, max_length=max_seq_length)

    # Initialize DistilBERT model
    num_labels = len(place_reviews['sentiment'].unique())
    model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

    # Compile the model with adjusted hyperparameters
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)  # Adjust the learning rate
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    # Increase the number of epochs for better training
    epochs = 1  # Increase the number of epochs

    # Reduce batch size for better memory utilization
    batch_size = 8  # You can adjust this value based on available memory

    # Model training
    history = model.fit(
        x={'input_ids': x_train_encoded['input_ids'], 'attention_mask': x_train_encoded['attention_mask']},
        y=y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(
            {'input_ids': x_test_encoded['input_ids'], 'attention_mask': x_test_encoded['attention_mask']},
            y_test
        )
    )
    model_tot[place]=model

    # Model evaluation
    y_pred = np.argmax(model.predict({'input_ids': x_test_encoded['input_ids'], 'attention_mask': x_test_encoded['attention_mask']}).logits, axis=1)
    accuracy = np.mean(y_pred == y_test)
    unique_classes = df['sentiment'].unique()
    classification_rep = classification_report(y_test, y_pred, labels=unique_classes, target_names=unique_classes, output_dict=True)

    y_pred_tot[place]=y_pred
    accuracy_tot[place]=accuracy
    classification_rep_tot[place]=classification_rep

# Example usage:


<ipython-input-19-a373e3dd0fd1>:20: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


Processing reviews for Place: Charminar


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

2/2 [==============================] - 2s 29ms/step
Processing reviews for Place: Golconda Fort


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of the PyTorch model were not used when initializin

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of the PyTorch model were not used when initializin

1/1 [==============================] - 3s 3s/step
Processing reviews for Place: NTR Garden


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of the PyTorch model were not used when initializin

2/2 [==============================] - 2s 36ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
print(model_tot)

{'Charminar': <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x7c46f5f57160>, 'Golconda Fort': <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x7c46eef7ea70>, 'Wonderla': <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x7c47066f5090>, 'Ramoji Film City': <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x7c48c2370a00>, 'Nehru Zoological Park': <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x7c46ed1ee200>, 'Birla Science Museum': <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x7c46ed1ee8f0>, 'Hussain Sagar Lake': <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x7c47f87b0850>, 'Birla mandir': <transform

In [21]:
y_pred_tot

{'Charminar': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'Golconda Fort': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'Wonderla': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'Ramoji Film City': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'Nehru Zoological Park': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'Birla Science Museum': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]),
 'Hussain Sagar Lake': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'Birla mandir': array([1, 1, 1, 1, 1, 1, 1, 1, 

In [22]:
accuracy_tot

{'Charminar': 0.696969696969697,
 'Golconda Fort': 0.9393939393939394,
 'Wonderla': 1.0,
 'Ramoji Film City': 0.8787878787878788,
 'Nehru Zoological Park': 0.8333333333333334,
 'Birla Science Museum': 0.7916666666666666,
 'Hussain Sagar Lake': 0.7777777777777778,
 'Birla mandir': 0.9166666666666666,
 'Cable Bridge': 0.42857142857142855,
 'NTR Garden': 0.75}

In [ ]:
def place_process(place,rating):
    place_temp = df[df['place_visit'] == place]
    partner_counts=place_temp['partner'].value_counts()
    max_partner = partner_counts.idxmax()
    likely_partner[place]=max_partner

    time_counts=place_temp['time'].value_counts()
    max_time = time_counts.idxmax()
    likely_time[place]=max_time

    sum_ratings = place_temp['Rating'].sum()
    num_entries = place_temp.shape[0]
    average_rating = sum_ratings / num_entries

    avg_rating[place]=average_rating

In [25]:
def predict_emotion(review, place):
    if place not in place_sentiments:
        return "Place not found"

    inp_model = model_tot[place]
    #tokenizer remains same
    #label encoder remains same

    # Tokenize and encode the review

    review_encoded = tokenizer([preprocess_text(review)], return_tensors='tf', padding=True, truncation=True, max_length=128)
    print(review_encoded)
    # Predict sentiment
    sentiment = inp_model.predict({'input_ids': review_encoded['input_ids'], 'attention_mask': review_encoded['attention_mask']}).logits
    sentiment_label = label_encoder.inverse_transform([np.argmax(sentiment)])[0]

    return sentiment_label

In [34]:
user_review = "This place is worst. It is really bad and crowded"
place_name = "Wonderla"
predicted_emotion = predict_emotion(user_review, place_name)
print(f"Predicted emotion for '{user_review}' at '{place_name}': {predicted_emotion}")

{'input_ids': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=
array([[  101,  2023,  2173,  2003,  5409,  2009,  2003,  2428,  2919,
         1998, 10789,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
1/1 [==============================] - 0s 49ms/step
Predicted emotion for 'This place is worst. It is really bad and crowded' at 'Wonderla': Happy
